#  Классификация текстов с использованием эмбеддингов слов.

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://pytorch.org/text/stable/vocab.html
* https://pytorch.org/text/stable/transforms.html
* https://rusvectores.org/
* https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html
* https://github.com/natasha/navec
* https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html
* https://torchmetrics.readthedocs.io/en/stable/

## Задачи для совместного разбора

1\. Реализуйте модель для классификации текстов с использованием слоя `nn.Embedding`. Заморозьте веса слоя эмбеддингов.

In [ ]:
import torch as th
import torch.nn as nn

In [ ]:
X = th.randint(0, 1000, size=(16, 20)).long()
y = th.LongTensor([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.emb = nn.Embedding(
        num_embeddings=1000,
        embedding_dim=100,
    )
    self.fc = nn.Linear(in_features=100, out_features=2)

  def forward(self, X):
    # X: batch_size x seq_len
    e = self.emb(X) # batch_size x seq_len x emb_dim
    # преобразовать эмб. токенов в эмб. последовательности
    e = e.mean(dim=1) # batch_size x emb_dim
    out = self.fc(e) # batch_size x 2
    return out

In [ ]:
model = Net()
y_pred = model(X)
y_pred.shape

torch.Size([16, 2])

In [ ]:
model.emb.weight.requires_grad_(False)
model.emb.weight.requires_grad

False

2\. Используя `torchmetrics`, рассчитайте значение accuracy по эпохам с использованием мини-пакетного градиентого спуска.

In [ ]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 10.3 MB/s eta 0:00:00


In [ ]:
import torchmetrics as M

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim

dset = TensorDataset(X, y)
loader = DataLoader(dset, batch_size=4)

n_epochs = 5
lr = 0.001

model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(n_epochs):
  preds, trues = [], []
  running_correct, running_count = 0, 0
  running_acc, running_steps = 0, 0

  acc_m = M.Accuracy(task="binary")

  for step, (X_b, y_b) in enumerate(loader):
    y_pred = model(X_b) # batch_size x n_classes
    loss = criterion(y_pred, y_b)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # v1
    preds.extend(list(y_pred.argmax(dim=1)))
    trues.extend(list(y_b))

    # v2
    running_correct += (y_pred.argmax(dim=1) == y_b).sum()
    running_count += len(y_b)

    # v3
    acc = (y_pred.argmax(dim=1) == y_b).float().mean()
    running_acc += acc
    running_steps += 1

    # v4
    acc_m.update(y_pred.argmax(dim=1), y_b)

    print(f"{epoch=} {step=} {acc.item()=}")

  preds = th.tensor(preds)
  trues = th.tensor(trues)
  acc_epochs = (preds == trues).float().mean()
  acc_epochs2 = running_correct / running_count
  acc_epochs3 = running_acc / running_steps
  acc_epochs4 = acc_m.compute()
  print(f"{epoch=} "
        f"{acc_epochs.item()=} \n"
        f"{acc_epochs2.item()=} \n"
        f"{acc_epochs3.item()=} \n"
        f"{acc_epochs4.item()=}"
  )

epoch=0 step=0 acc.item()=0.25
epoch=0 step=1 acc.item()=0.5
epoch=0 step=2 acc.item()=0.5
epoch=0 step=3 acc.item()=0.5
epoch=0 acc_epochs.item()=0.4375 
acc_epochs2.item()=0.4375  
acc_epochs3.item()=0.4375  
acc_epochs4.item()=0.4375
epoch=1 step=0 acc.item()=0.5
epoch=1 step=1 acc.item()=0.5
epoch=1 step=2 acc.item()=0.5
epoch=1 step=3 acc.item()=0.5
epoch=1 acc_epochs.item()=0.5 
acc_epochs2.item()=0.5  
acc_epochs3.item()=0.5  
acc_epochs4.item()=0.5
epoch=2 step=0 acc.item()=0.5
epoch=2 step=1 acc.item()=0.75
epoch=2 step=2 acc.item()=0.5
epoch=2 step=3 acc.item()=0.5
epoch=2 acc_epochs.item()=0.5625 
acc_epochs2.item()=0.5625  
acc_epochs3.item()=0.5625  
acc_epochs4.item()=0.5625
epoch=3 step=0 acc.item()=0.5
epoch=3 step=1 acc.item()=0.75
epoch=3 step=2 acc.item()=0.5
epoch=3 step=3 acc.item()=0.5
epoch=3 acc_epochs.item()=0.5625 
acc_epochs2.item()=0.5625  
acc_epochs3.item()=0.5625  
acc_epochs4.item()=0.5625
epoch=4 step=0 acc.item()=0.5
epoch=4 step=1 acc.item()=0.75
epoc

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Считайте файл `lenta_news.csv` и разбейте на обучающую и тестовую выборку. Выполните предобработку текста и создайте Vocab на основе обучающей выборки (токен - слово). Выведите на экран количество токенов в полученном словаре.

- [ ] Проверено на семинаре

<p class="task" id="2"></p>


2\. Создайте класс `NewsDataset`. Реализуйте метод `__getitem__` таким образом, чтобы он возвращал набор индексов токенов для текста новости (или новостей, если используются срезы) и метки классов для этих новостей. Используя преобразования, сделайте длины наборов индексов одинаковой фиксированной длины (подходящее значение определите сами). Закодируйте целыми числами категории новостей. Создайте два объекта класса `NewsDataset` (для обучающей и тестовой выборки).

Выведите на экран результат выполнения `train_dataset[0]` и `train_dataset[:3]`

- [ ] Проверено на семинаре

<p class="task" id="3"></p>

3\. Реализуйте модель, которая получает на вход батч новостей (в виде индексов токенов), пропускает его через слой `nn.Embedding` (матрица эмбеддингов инициализируется случайным образом), после чего передает полученные эмбеддинги части-классификатору (который состоит из некоторого количества полносвязных слоев). Для получения эмбеддинга для предложения на основе эмбеддингов слов воспользуйтесь любой функцией агрегации, сохраняющей размерности векторов (сумма, усреднение и т.д.).

Решите задачу классификации новостей. Постройте график изменения значения функции потерь на обучающем множестве в зависимости от номера эпохи, графики изменения метрики f1 на обучающем и тестовом множестве в зависимости от эпохи. Выведите на экран отчет по классификации на обучающем и тестовом множестве.

- [ ] Проверено на семинаре

<p class="task" id="4"></p>

4\. Повторите решение задачи 3, создав слой `nn.Embedding` на основе предобученных векторов для слов русского языка и заморозив веса данного слоя. Для поиска векторов можете воспользоваться любым известным вам ресурсом. Сравните качество полученного решения и решения из предыдущей задачи, а также время, затраченное на обучения моделей.  

- [ ] Проверено на семинаре

<p class="task" id="5"></p>

5\. Повторите решение задачи 3, не замораживая веса слоя эмбеддингов. Сравните качество полученного решения и решений из предыдущих задач, а также время, затраченное на обучения моделей.  

- [ ] Проверено на семинаре

<p class="task" id="6"></p>

6\. Воспользовавшись обученной моделью из предыдущей задачи, визуализируйте эмбеддинги новостей из тестовой выборки в двумерном пространстве. Для проекции точек в двумерное пространство воспользуйтесь алгоритмом t-SNE. Раскрасьте точки в цвет, соответствующий классу новости.

- [ ] Проверено на семинаре

## Обратная связь
- [ ] Хочу получить обратную связь по решению